<a href="https://colab.research.google.com/github/MedicalImageAnalysisTutorials/DeepLearning4All/blob/main/IA_DNN_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Generative Adversarial Network (GAN)**

## Introduction

A generative adversarial network (GAN) is a class of machine learning frameworks designed by Ian Goodfellow and his colleagues in 2014.
Generative Adversarial Networks, or GANs for short, are an approach to generative modeling using deep learning methods, such as convolutional neural networks.

Generative modeling is an unsupervised learning task in machine learning that involves automatically discovering and learning the regularities or patterns in 
input data in such a way that the model can be used to generate or output new examples that plausibly could have been drawn from the original dataset.


GAN has two parts:

The generator learns to generate plausible data. The generated instances become negative training examples for the discriminator.
The discriminator learns to distinguish the generator's fake data from real data. The discriminator penalizes the generator for producing implausible results.

1.When training begins, the generator produces obviously fake data, and the discriminator quickly learns to tell that it's fake.

2.As training progresses, the generator gets closer to producing output that can fool the discriminator.

3.Finally, if generator training goes well, the discriminator gets worse at telling the difference between real and fake. It starts to classify fake data as real, and its accuracy decreases.

### **Applications**

1.**Data Augmentation**: Data augmentation in data analysis are techniques used to increase the amount of data by adding slightly modified copies of already existing data or newly created synthetic data from existing data.
Medical Application

![](https://machinelearningmastery.com/wp-content/uploads/2019/04/Example-of-Vector-Arithmetic-for-GAN-Generated-Faces.png)

2.**Image Translation**


![](https://machinelearningmastery.com/wp-content/uploads/2019/06/Example-of-Sketches-to-Color-Photographs-with-pix2pix.png)


**Some more applications are**:


Medical Application

Generate Examples for Image Datasets

Generate Photographs of Human Faces

Generate Realistic Photographs

Generate Cartoon Characters

Image-to-Image Translation

Text-to-Image Translation

Semantic-Image-to-Photo Translation

Face Frontal View Generation

Generate New Human Poses

Photos to Emojis

Photograph Editing

Face Aging

Photo Blending

Super Resolution

Photo Inpainting

Clothing Translation

Video Prediction

3D Object Generation






###**Architecture**
A picture of the whole system:

![](https://developers.google.com/machine-learning/gan/images/gan_diagram.svg)

###**Loss function**

**Discriminator:**
$$
D_{loss} = \frac{1}{m} \sum_{i=1}^{m} [\log{D(x^{(i)})} + \log{(1-D(G(z^{(i)})))}]
$$

**Generator:**
$$
G_{loss} = \frac{1}{m} \sum_{i=1}^{m} \log{(1-D(G(z^{(i)})))}
$$

where $D$ is the discriminator and $G$ is the generator. $x^{(i)}$ is the $i$th ground truth data and $z^{(i)}$ is the $i$th generated data(random input). $m$ is the total number of data.

The discriminator is to find the mistake and the generator wants to fake the discriminator.

If $D()$ is 1 means the discriminator thinks the data is real. if $D()$ is 0 means it is fake. But note that $\log{0}$  is negative infinity.

We want to max the loss function for discriminator and minimize the loss function for generator. Loss function close to 0 is max. Loss function close to negative infinity is min.

Combining the two losses together,
$$\min_{G}\max_{D} V(D,G) = E_x[\log{D(x)}] + E_z[\log{(1-D(G(z)))}]$$

where $E_x$ is the expected value over all real data instances. $E_z$ is the expected value over all random inputs to the generator.

In practice, we change the generator loss function to 
$$ G_{loss} = \max_{G} E_z[\log{D(G(z))}]$$

Discriminator loss function:
$$\max_{D} V(D,G) = E_x[\log{D(x)}] + E_z[\log{(1-D(G(z)))}]$$

# Notebook Setup 

In [11]:
# Preparing 2d datset from medical images (ct to mri and mri to ct) 
# download medical image dataset use only t2 and ct
# unzip, load to slicer then save as nrrd
# reset the scene and load the nrrd files
# register (low to high) e.g. t2  to ct
# save with same name of the moving image and add "_reg.nrrd"
# this is our 3d dataset
# during training: 
#   img = read using simpleitk 
#   imgArr = convert to array 
#   this array is now your 2d datset e.g. 28, 512,512 
# Setup 
usePt = 1
doInstall =1
if doInstall:
  !pip install SimpleITK
# !pip install albumentations
!pip install albumentations==0.4.6
import albumentations 
from albumentations.pytorch import ToTensorV2
import os, random, time, math
import numpy as np
import matplotlib.pyplot as plt
import albumentations as A
import cv2 
import SimpleITK as sitk 
import tensorflow as tf
from tensorflow import keras
from PIL import Image

if usePt == 0:
  from tensorflow.keras import layers
  from tensorflow.keras import datasets, layers, models

if usePt == 1:
  import torch
  import torch.nn as nn
  import torch.optim as optim
  import torchvision
  # import torchvision.datasets as datasets
  from torchvision import datasets
  from torch.utils.data import DataLoader, Dataset
  import torchvision.transforms as transforms
  from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard
  from torchvision import transforms


# to reproduce the same results given same input
np.random.seed(1)               


# GAN Hyper Parameters

In [12]:
# notebook parameters
if usePt == 1:
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
doAug = 0
GANs = ['simple', 'DCGAN', 'WGAN', 'P2P'] # 0, 1, 2, 3
GANID = 3
datasetID = 5  # 1:minst is selected by default, for cifar10 use 2
               # 5: mr-ct dataset
NNID      = 3  # 1:NN is by default, for DNN use 2,or 3, for 3D use 4  
number_of_classes = 10  # each datasets have 10 classes
showSamples = 1
UseOurDatasets = 0
new_size = (64,64)
# GAN parameters
LEARNING_RATE = 2e-4  # could also use two lrs, one for gen and one for dsc
BATCH_SIZE = 32
IMAGE_SIZE = 64
CHANNELS_IMG = 1 # channels
Z_DIM = 100 # 
NUM_EPOCHS = 5 
FEATURES_DISC = 64 
FEATURES_GEN = 64 
CRITIC_ITERATIONS = 5
WEIGHT_CLIP = 0.01

# if you have large GPU memory you can combine the images to batches 
# for faster training.
# It is good to try different values
batch_size = 32 # you can try larger batch size e.g. 1024 * 6

# Dataset preparation

### Datasets

In [13]:
# $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ Using MNIST
from albumentations.pytorch import ToTensorV2
def datasetReturn(GANID):
  import torchvision.transforms as transforms
  if GANID == 0:
    transforms = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)),]
    )
  else:
    transforms = transforms.Compose([transforms.Resize(IMAGE_SIZE),transforms.ToTensor(),transforms.Normalize([0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)]),])

  dataset = datasets.MNIST(root="dataset/", transform=transforms, download=True)
  return dataset
  
both_transform = A.Compose(
    [A.Resize(width=256, height=256),], additional_targets={"image0": "image"},
)

transform_only_input = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], max_pixel_value=255.0,),
        ToTensorV2(),
    ]
)
transform_only_mask = A.Compose(
    [
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], max_pixel_value=255.0,),
        ToTensorV2(),
    ]
)


# $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ Using our own datasets
class MedicalDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None): 
        self.root_dir = root_dir   
        self.transform = transform 
        self.images = os.listdir(self.root_dir)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self,index):#
        image_index = self.images[index]# Using index to get image
        img_path = os.path.join(self.root_dir, image_index)# get path
        img = io.imread(img_path)# read image
        label = img_path.split('\\')[-1].split('.')[0]# get label
        sample = {'image':img,'label':label}


class MapDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.list_files = os.listdir(self.root_dir)

    def __len__(self):
        return len(self.list_files)

    def __getitem__(self, index):
        img_file = self.list_files[index]
        img_path = os.path.join(self.root_dir, img_file)
        image = np.array(Image.open(img_path))
        input_image = image[:, :600, :]
        target_image = image[:, 600:, :]

        augmentations = both_transform(image=input_image, image0=target_image)
        input_image = augmentations["image"]
        target_image = augmentations["image0"]

        input_image = transform_only_input(image=input_image)["image"]
        target_image = transform_only_mask(image=target_image)["image"]

        return input_image, target_image


# if UseOurDa
# data = AnimalData('E:/Python Project/PyTorch/dogs-vs-cats/train',transform=None)


# # if __name__=='__main__':
#     data = AnimalData('E:/Python Project/PyTorch/dogs-vs-cats/train',transform=None)
#     dataloader = DataLoader(data,batch_size=BATCH_SIZE,shuffle=True)
#     for i_batch,batch_data in enumerate(dataloader):
#         print(i_batch)
#         print(batch_data['image'].size())
#         print(batch_data['label'])



In [14]:
# !wget --no-check-certificate https://www.kaggle.com/vikramtiwari/pix2pix-dataset?select=cityscapes -O ctcscapes-datasets.tar
# !unzip ct-mr-datasets.zip

In [15]:
def read3DMI(imgPath,doNormalisation=1, new_size=[]):
    img = sitk.ReadImage(imgPath)
    # print(img.GetSize())
    imga = sitk.GetArrayFromImage(img) # sitk reverse the 3rd dimension
    # print(img.GetSize())
    # print(imga.shape)
    if len(new_size)>0:      
        imgaNew= [] #np.zeros(new_size)
        # resize image
        for i in range (imga.shape[0]):
            s = imga[i,:,:]
            v = cv2.resize(s, (64,64), interpolation = cv2.INTER_AREA)
            imgaNew.append(v) #cv2.resize(imga[i,:,:], new_size, interpolation = cv2.INTER_AREA))
        imga = np.array(imgaNew)  

    if doNormalisation:
       imgMax = np.max(imga) 
       imgMin = np.min(imga) 
       imga = (imga - imgMin) / (imgMax-imgMin)
            
    return img, imga

# minst dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
class_names = range(10)
if datasetID==2:
    # cifar10 dataset
    # The CIFAR10 dataset contains 60,000 color images in 10 classes, 
    # with 6,000 images in each class.
    # The dataset is divided into 50,000 training images and 10,000 testing images.
    # The classes are mutually exclusive and there is no overlap between them.

    (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
    class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
    if NNID==4:
        #TODO: fix this 
        showSamples =0
        
        x_train = x_train.reshape(-1,32*32*3) # (32 * 32 * 3)        
        x_train = np.resize(x_train,(x_train.shape[0],15,15,15))        
        x_train = x_train.reshape(-1,15,15,15)
        x_test = x_test.reshape(-1,32*32*3) # (32 * 32 * 3)
        x_test = np.resize(x_test,(x_test.shape[0],15,15,15))
        x_test = x_test.reshape(-1,15,15,15)
        x_train  =  x_train[..., np.newaxis] # np.reshape(x_train, (-1, h,w,1))
        y_train  =  y_train[..., np.newaxis] # np.reshape(y_train, (-1, h,w,1))
        x_test   =  x_test[..., np.newaxis]  # np.reshape(x_test,  (-1, h,w,1))
        y_test   =  y_test[..., np.newaxis]  # np.reshape(y_test,  (-1, h,w,1))


        print(x_train.shape)
        print(x_test.shape)
elif datasetID==5:
    #  !wget --no-check-certificate https://cloud.uni-koblenz.de/s/GS8YKpC83A6jLr8/download/mri_ct_images.zip -O ct-mr-datasets.zip
    #  !unzip ct-mr-datasets.zip
     fnms = sorted(os.listdir("Registration_image"))
     ctFnms = [x for x in fnms if "ct" in x]
     mrFnms = [x for x in fnms if "reg" in x]
     print("fnms : ",len(fnms))
     print("ctFnms　: ",len(mrFnms))
     print(ctFnms)
     print(mrFnms)
     ct_dataset = []
     mr_dataset = []
     num_ct = 0
     num_mri = 0
    #  img,imgsCT =  read3DMI("/content/Registration_image/"+ctFnms[0],doNormalisation=1, new_size=(64,64))
    #  patient_000_ct 
    #  patient_000_mr_T2_reg
     for i in range(len(mrFnms)):
        img,imgsCT =  read3DMI("/content/Registration_image/"+ctFnms[i],doNormalisation=1, new_size=new_size)
        img,imgsMR =  read3DMI("/content/Registration_image/"+mrFnms[i],doNormalisation=1, new_size=new_size)
        # 29, 512 x 512   ---> num_2d, 64,64        
        # 40, 256 x 256   ---> num_2d, 64,64
        # ...
        # 
        # sum(num_2d),64,64     :X=ct,Y=mri
        #
        num_ct  += imgsCT.shape[0] 
        num_mri += imgsCT.shape[0] 
        ct_dataset.extend(imgsCT)
        mr_dataset.extend(imgsMR)
     print(num_mri) # 238
     print(num_ct)
     ct_dataset = np.array(ct_dataset)
     mr_dataset = np.array(mr_dataset)
     ct_dataset = ct_dataset[..., np.newaxis]
     mr_dataset = mr_dataset[..., np.newaxis]

     train_CT  = np.array(ct_dataset)
     train_mri = np.array(mr_dataset)
      # Convert to tensor 
     train_CT  = torch.from_numpy(train_CT)
     train_mri = torch.from_numpy(train_mri)
     print(train_CT.shape)
      # get size 
     h, w = 64, 64
    #  print(ok)
    # check for rgb 
    #dataset = torch.from_numpy()



try:
    # number of channels
    c =  x_train.shape[3]
except:
    # number of channels
    c =  1
    # if there is no number of channels, add 1
    x_train  =  x_train[..., np.newaxis] # np.reshape(x_train, (-1, h,w,1))
    y_train  =  y_train[..., np.newaxis] # np.reshape(y_train, (-1, h,w,1))
    x_test   =  x_test[..., np.newaxis]  # np.reshape(x_test,  (-1, h,w,1))
    y_test   =  y_test[..., np.newaxis]  # np.reshape(y_test,  (-1, h,w,1))


# Reserve 10,000 samples for validation.
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]
h, w = 64, 64
number_of_pixels = h * w * c


print("dataset shape   : ",x_train.shape)
print("number of images: ",x_train.shape[0])
print("image size      : ",x_train[0].shape)
print("image data type : ",type(x_train[0][0][0][0]))
print("image max  value: ",np.max(x_train[0]))
print("image min  value: ",np.min(x_train[0]))
if c==1:
   print("gray or binary image (not color image)")
elif c==3:
   print("rgb color image (or probably non-color image represented with 3 channels)")


# display sample images 
if showSamples:
    plt.figure(figsize=(10,10))
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        #plt.imshow(x_train[i])
        print('kkkkkkkkkkkkkkkkkkkkkkkkkk', type(x_train[i]))
        plt.imshow(cv2.cvtColor(x_train[i], cv2.COLOR_BGR2RGB))

        # The CIFAR labels happen to be arrays, 
        # which is why you need the extra index
        if datasetID==1:
            plt.xlabel(y_train[i])
        elif datasetID==2:
            plt.xlabel(class_names[y_train[i][0]])
    plt.show()


# normalisation
x_train = np.array([ x/255.0 for x in x_train])
x_val   = np.array([ x/255.0 for x in x_val])
x_test  = np.array([ x/255.0 for x in x_test])
#y_train = y_train.astype(np.float32)

# for NN we need 1D 
if NNID ==1:
   x_train = np.reshape(x_train, (-1, number_of_pixels))
   x_val   = np.reshape(x_val,  (-1, number_of_pixels))
   x_test  = np.reshape(x_test , (-1, number_of_pixels))
print(x_train.shape,y_train.shape)
# x_train = np.reshape(x_train, (50000, 64, 64, 1))
# x_train = np.resize(x_train[:,])

# Prepare the training dataset.
print(x_train.shape,y_train.shape)
# print(ok)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)
#train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

# Prepare the test dataset.
tst_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
tst_dataset = tst_dataset.batch(batch_size)

FileNotFoundError: ignored

In [ ]:
fnms = sorted(os.listdir("Registration_image"))
ctFnms = [x for x in fnms if 'ct' in x]
ctFnms

## Dataset augmentation

It is important to train the model on different variations of the dataset. It is also important to have large datset for training.

Using dataset augmentation helps to achieve both of the above goals. From one image, one can generate hundred thousands of images using image transformation.

The image transformation could be [spatial transform]() or point transform where we move the points of the image to new locations e.g. shifting, flipping, and/or rotating the imag. 

Another type of transformation is intensity transform or pixel transform where we change the color values of the pixels in the image e.g. invert the color, add more brightness or darkness. 



In [ ]:
def imagePixelTransforms(img):    
    images = []
    # let's make 3 simple transformations
    img1   = 1.0- img # invert color
    img2   = img +0.3 # more brightness
    img3   = img -0.3 # more darkness
    images = np.array([img1,img2,img3])
    images = [ img.reshape(img.shape) for img in images]

    # plt.figure() ;    plt.imshow(img)
    # plt.figure() ;    plt.imshow(img1)
    # plt.figure() ;    plt.imshow(img2)
    # plt.figure() ;    plt.imshow(img3)    
    return images

def imagePointTransforms(img):
    images = []
    # let's make 3 simple transformations
    # Perform the rotation
    center  = (img.shape[0] / 2, img.shape[1] / 2)
    sz      = (img.shape[1], img.shape[0])
    tMatrix = cv2.getRotationMatrix2D(center, 45, 1)
    img1 = cv2.warpAffine(img, tMatrix, sz)
    img1 = img1[...,np.newaxis] if img1.shape !=img.shape else img1
    tMatrix = cv2.getRotationMatrix2D(center, 90, 1)
    img2 = cv2.warpAffine(img, tMatrix, sz)
    img2 = img2[...,np.newaxis] if img2.shape !=img.shape else img2
    tMatrix = cv2.getRotationMatrix2D(center, 270, 1)
    img3 = cv2.warpAffine(img, tMatrix, sz)
    img3 = img3[...,np.newaxis] if img3.shape !=img.shape else img3

    images = np.array([img1,img2,img3])
    # plt.figure() ;    plt.imshow(img)
    # plt.figure() ;    plt.imshow(img1)
    # plt.figure() ;    plt.imshow(img2)
    # plt.figure() ;    plt.imshow(img3)    
    #plt.figure() ;    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    # plt.figure() ;    plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
    return images


# define a function for sitk transform
def resample(img_array, transform):
    # Output image Origin, Spacing, Size, Direction are taken from the reference
    # image in this call to Resample
    image = sitk.GetImageFromArray(img_array)
    reference_image = image
    interpolator = sitk.sitkCosineWindowedSinc
    default_value = 100.0
    resampled_img = sitk.Resample(image, reference_image, transform,
                         interpolator, default_value)
    resampled_array = sitk.GetArrayFromImage(resampled_img)
    return resampled_array

def affine_rotate(transform, degrees):
    parameters = np.array(transform.GetParameters())
    new_transform = sitk.AffineTransform(transform)
    dimension =3 
    matrix = np.array(transform.GetMatrix()).reshape((dimension,dimension))
    radians = -np.pi * degrees / 180.
    rotation = np.array([[1  ,0,0], 
                         [0, np.cos(radians), -np.sin(radians)],
                         [0, np.sin(radians), np.cos(radians)]]
                        )
    new_matrix = np.dot(rotation, matrix)
    new_transform.SetMatrix(new_matrix.ravel())
    return new_transform


def imagePoint3DTransforms(img):
    #print("imagePoint3DTransforms")
    images = []
    # let's make 3 simple transformations
    # Perform the rotation
    # In SimpleITK resampling convention, the transformation maps points 
    # from the fixed image to the moving image,
    # so inverse of the transform is applied

    center = (img.shape[0] /2, img.shape[1] /2,img.shape[1] /2)
    rotation_around_center = sitk.AffineTransform(3)
    rotation_around_center.SetCenter(center)
    
    rotation_around_center = affine_rotate(rotation_around_center, -45)
    img1 = resample(img, rotation_around_center)

    rotation_around_center = affine_rotate(rotation_around_center, -90)
    img2 = resample(img, rotation_around_center)

    rotation_around_center = affine_rotate(rotation_around_center, -90)
    img3 = resample(img, rotation_around_center)

    images = np.array([img1,img2,img3])
    # plt.figure() ;    plt.imshow(img)
    # plt.figure() ;    plt.imshow(img1)
    # plt.figure() ;    plt.imshow(img2)
    # plt.figure() ;    plt.imshow(img3)    
    #plt.figure() ;    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    # plt.figure() ;    plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
    return images

def doAugmentation(images,labels,batch_size):
    # input is an image or a batch e.g. list of images 
    # get numpy arrays from the tensor    
    images = images.numpy()
    labels = labels.numpy()
    # if 1d convert back to 2d
    #print(images.shape)
    rgb = 0 ; is3d = 0
    if len(images.shape) == 2:
       try: 
          img2d_shape = int(math.sqrt(images.shape[1])) # gray or binary image
          images =images.reshape(-1,img2d_shape,img2d_shape)
       except:
          try: 
            img2d_shape = int(math.sqrt(images.shape[1]/3)) # rgb image
            images =images.reshape(-1,img2d_shape,img2d_shape,3)
            rgb = 1  
          except:
            pass  
            # img3d_shape = int(math.sqrt(images.shape[1]/3)) # rgb image
            # images =images.reshape(-1,img2d_shape,img2d_shape,3)
            # is3d = 1  



    x_outputs = [] ; y_outputs = []
    i = 0
    for img in images:
        #print("-------------------------", i ,"--------------------")
        if NNID==4:
           img = img.squeeze() 
        # from each images we generate 6 images
        # 64 batch will generate 448
        x_outputs.extend([img])
        imgs1 = imagePoint3DTransforms(img)
        imgs2 = imagePixelTransforms(img)
        #if not rgb:
           #imgs1 = np.array( x[...,np.newaxis] for x in imgs1 if len(x.shape)<3) 
           #imgs2 = np.array( x[...,np.newaxis] for x in imgs2 if len(x.shape)<3)
        x_outputs.extend(imgs1) # 3 images
        x_outputs.extend(imgs2) # 3 images
        # print(img.shape)
        # print(imgs1[0].shape)
        # print(imgs2[0].shape)
        # assign the same label to all transformed images
        for j in range ( len(imgs1) +len(imgs2)+1):
            y_outputs.extend([labels[i]])

        i = i +1
    x_outputs = np.array(x_outputs)
    if NNID==4:
       x_outputs = np.array([x[...,np.newaxis] for x in x_outputs])
    y_outputs = np.array(y_outputs)

    if (not rgb) and (NNID==1):
       x_outputs = np.reshape(x_outputs, (-1,img2d_shape*img2d_shape,1))
    elif (rgb) and (NNID==1):
       x_outputs = np.reshape(x_outputs, (-1,img2d_shape*img2d_shape*3))   

    new_train_dataset = tf.data.Dataset.from_tensor_slices((x_outputs, y_outputs))
    new_train_dataset = new_train_dataset.shuffle(buffer_size=1024).batch(batch_size)

    return new_train_dataset

##  1 Simple GAN model

this is basic one-hiden layer neural network without concolution involved

### Implementation

In [ ]:
if usePt == 0:
  discriminator_tf_simple = keras.Sequential(
      [
      keras.Input(shape = (784,)), # 784  #28
      layers.Dense(128),
      layers.LeakyReLU(0.01),
      layers.Dense(1, activation='sigmoid'),
      #  layers.Sigmoid(),
      ],
      name = "discriminator_tf",
  )

  generator_tf_simple = keras.Sequential(
      [
          keras.Input(shape=(64,)),
          layers.Dense(256),
          # layers.Reshape((8, 8, 128)),
          layers.LeakyReLU(alpha=0.01),
          layers.Dense(784, activation='tanh'), # 28x28
          # layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid"),
      ],
      name="generator_tf",
  )

  criterion = keras.losses.BinaryCrossentropy()
  opt_gen = keras.optimizers.Adam(3e-4)
  opt_dis = keras.optimizers.Adam(3e-4)
else:
    # Pytorch
    class SimpleDiscriminator(nn.Module):
        def __init__(self, in_features):
            super().__init__()
            print(in_features)
            self.disc = nn.Sequential(
                nn.Linear(in_features, 128),
                nn.LeakyReLU(0.01),
                nn.Linear(128, 1),
                nn.Sigmoid(),
            )

        def forward(self, x):
            return self.disc(x)

    class SimpleGenerator(nn.Module):
        def __init__(self, z_dim, img_dim):
            super().__init__()
            self.gen = nn.Sequential(
                nn.Linear(z_dim, 256),
                nn.LeakyReLU(0.01),
                nn.Linear(256, img_dim),
                nn.Tanh(),  # normalize inputs to [-1, 1] so make outputs [-1, 1]
            )

        def forward(self, x):
            return self.gen(x)

    # optimisers             
    # opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE)
    # opt_gen  = optim.Adam(gen.parameters(), lr=LEARNING_RATE)
    
    # loss function
    criterion = nn.BCELoss()


## 2 DCGAN(Deep Convolutional Generative Adversarial Networks)

### Introduction

In recent years, supervised learning with convolutional networks (CNNs) has seen huge adoption in computer vision applications. Comparatively, unsupervised learning with CNNs has received less attention. In this work we hope to help bridge the gap between the success of CNNs for supervised learning and unsupervised learning. We introduce a class of CNNs called deep convolutional generative adversarial networks (DCGANs), that have certain architectural constraints, and demonstrate that they are a strong candidate for unsupervised learning.

DCGAN uses convolutional and convolutional-transpose layers in the generator and discriminator, respectively. Here the discriminator consists of strided convolution layers, batch normalization layers, and LeakyRelu as activation function. It takes a 3x64x64 input image. The generator consists of convolutional-transpose layers, batch normalization layers, and ReLU activations. The output will be a 3x64x64 RGB image

![](https://editor.analyticsvidhya.com/uploads/2665314.png)

Figure:DCGAN generator used for LSUN scene modeling. A 100 dimensional uniform distribution Z is projected to a small spatial extent convolutional representation with many feature maps.
A series of four fractionally-strided convolutions (in some recent papers, these are wrongly called
deconvolutions) then convert this high level representation into a 64 × 64 pixel image. Notably, no
fully connected or pooling layers are used.

Referance:https://arxiv.org/pdf/1511.06434.pdf

paper link: https://arxiv.org/abs/1511.06434

### Implementation

In [ ]:
## tensorflow

In [ ]:
#----------------------------------------------------------
#
#DCGAN model definition
#
#----------------------------------------------------------
## pytorch
"""
Discriminator and Generator implementation from DCGAN paper
"""
import torch
import torch.nn as nn
if usePt == 0:
  discriminator_tf_dc = keras.Sequential(
      [
        keras.Input(shape = (64,64,1)), # 784  #28
        layers.Conv2D(FEATURES_DISC, kernel_size = 4, strides=2, padding="same"),
        layers.LeakyReLU(0.2),
        layers.Conv2D(FEATURES_DISC*2, kernel_size = 4, strides=2, padding="same"),
        layers.LeakyReLU(0.2),
        layers.Conv2D(FEATURES_DISC*4, kernel_size = 4, strides=2, padding="same"),
        layers.LeakyReLU(0.2),
        layers.Conv2D(FEATURES_DISC*8, kernel_size = 4, strides=2, padding="same"),
        layers.LeakyReLU(0.2),layers.Flatten(),
        layers.Dropout(0.2),
        layers.Dense(1, activation="sigmoid"),
      ],
      name = "discriminator_tf_dc",
  )
  latent_dim = 128
  generator_tf_dc = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        layers.Dense(8 * 8 * 128),
        layers.Reshape((8, 8, 128)),
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(1, kernel_size=5, padding="same", activation="sigmoid"),
    ],
    name="generator",
)
else:
  class DCGAN_Discriminator(nn.Module):
      def __init__(self, channels_img, features_d):
          super(DCGAN_Discriminator, self).__init__()
          self.disc = nn.Sequential(
              # input: N x channels_img x 64 x 64
              nn.Conv2d(
                  channels_img, features_d, kernel_size=4, stride=2, padding=1
              ),
              nn.LeakyReLU(0.2),
              # _block(in_channels, out_channels, kernel_size, stride, padding)
              self._block(features_d, features_d * 2, 4, 2, 1),
              self._block(features_d * 2, features_d * 4, 4, 2, 1),
              self._block(features_d * 4, features_d * 8, 4, 2, 1),
              # After all _block img output is 4x4 (Conv2d below makes into 1x1)
              nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0),
              nn.Sigmoid(),
          )

      def _block(self, in_channels, out_channels, kernel_size, stride, padding):
          return nn.Sequential(
              nn.Conv2d(
                  in_channels,
                  out_channels,
                  kernel_size,
                  stride,
                  padding,
                  bias=False,
              ),
              #nn.BatchNorm2d(out_channels),
              nn.LeakyReLU(0.2),
          )

      def forward(self, x):
          return self.disc(x)


  class DCGAN_Generator(nn.Module):
      def __init__(self, channels_noise, channels_img, features_g):
          super(DCGAN_Generator, self).__init__()
          self.net = nn.Sequential(
              # Input: N x channels_noise x 1 x 1
              self._block(channels_noise, features_g * 16, 4, 1, 0),  # img: 4x4
              self._block(features_g * 16, features_g * 8, 4, 2, 1),  # img: 8x8
              self._block(features_g * 8, features_g * 4, 4, 2, 1),  # img: 16x16
              self._block(features_g * 4, features_g * 2, 4, 2, 1),  # img: 32x32
              nn.ConvTranspose2d(
                  features_g * 2, channels_img, kernel_size=4, stride=2, padding=1
              ),
              # Output: N x channels_img x 64 x 64
              nn.Tanh(),
          )

      def _block(self, in_channels, out_channels, kernel_size, stride, padding):
          return nn.Sequential(
              nn.ConvTranspose2d(
                  in_channels,
                  out_channels,
                  kernel_size,
                  stride,
                  padding,
                  bias=False,
              ),
              #nn.BatchNorm2d(out_channels),
              nn.ReLU(),
          )

      def forward(self, x):
          return self.net(x)

  # def test():
  #     N, in_channels, H, W = 8, 3, 64, 64
  #     noise_dim = 100
  #     x = torch.randn((N, in_channels, H, W))
  #     disc = Discriminator(in_channels, 8)
  #     assert disc(x).shape == (N, 1, 1, 1), "Discriminator test failed"
  #     gen = Generator(noise_dim, in_channels, 8)
  #     z = torch.randn((N, noise_dim, 1, 1))
  #     assert gen(z).shape == (N, in_channels, H, W), "Generator test failed"


# test()

## 3  WGAN(Wasserstein GAN)

### Introduction

The Wasserstein Generative Adversarial Network, or W-GAN, is an extension to the generative adversarial network that both improves the stability when training the model and provides a loss function that correlates with the quality of generated images.

The development of the WGAN has a dense mathematical motivation, although in practice requires only a few minor modifications to the established standard deep convolutional generative adversarial network, or DCGAN.

It is an extension of the GAN that seeks an alternate way of training the generator model to better approximate the distribution of data observed in a given training dataset.

Instead of using a discriminator to classify or predict the probability of generated images as being real or fake, the WGAN changes or replaces the discriminator model with a critic that scores the realness or fakeness of a given image.

This change is motivated by a theoretical argument that training the generator should seek a minimization of the distance between the distribution of the data observed in the training dataset and the distribution observed in generated examples.

The benefit of the WGAN is that the training process is more stable and less sensitive to model architecture and choice of hyperparameter configurations. Perhaps most importantly, the loss of the discriminator appears to relate to the quality of images created by the generator.


Referance Paper:
https://machinelearningmastery.com/how-to-code-a-wasserstein-generative-adversarial-network-wgan-from-scratch/

Resources and papers: \\
https://www.alexirpan.com/2017/02/22/... \\
https://arxiv.org/abs/1701.07875 \\
https://arxiv.org/abs/1704.00028 \\

### Implementation

In [ ]:
## tensorflow
##-------------------------------------------

##-------------------------------------------discriminator

def discriminator_tf_wgan(input, is_train):
    with tf.variable_scope('discriminator') as scope:
        # 64*64*64
        conv1 = tf.layers.conv2d(input, 64, kernel_size=5, strides=2, padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                 name='conv1')
        act1 = leaky_relu(conv1, n='act1')
 
        # 32*32*128
        conv2 = tf.layers.conv2d(act1, 128, kernel_size=5, strides=2, padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                 name='conv2')
        bn2 = tf.contrib.layers.batch_norm(conv2, is_training=is_train, epsilon=1e-5, decay=0.9,
                                           updates_collections=None, scope='bn2')
        act2 = leaky_relu(bn2, n='act2')
 
        # 16*16*256
        conv3 = tf.layers.conv2d(act2, 256, kernel_size=5, strides=2, padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                 name='conv3')
        bn3 = tf.contrib.layers.batch_norm(conv3, is_training=is_train, epsilon=1e-5, decay=0.9,
                                           updates_collections=None, scope='bn3')
        act3 = leaky_relu(bn3, n='act3')
 
        # 8*8*512
        conv4 = tf.layers.conv2d(act3, 512, kernel_size=5, strides=2, padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                 name='conv4')
        bn4 = tf.contrib.layers.batch_norm(conv4, is_training=is_train, epsilon=1e-5, decay=0.9, updates_collections=None,
                                           scope='bn4')
        act4 = leaky_relu(bn4, n='act4')
 
        # start from act4
        dim = int(np.prod(act4.get_shape()[1:]))
        fc1 = tf.reshape(act4, shape=[-1, dim], name='fc1')
        w2 = tf.get_variable('w2', shape=[fc1.shape[-1], 1], dtype=tf.float32,
                             initializer=tf.truncated_normal_initializer(stddev=0.02))
        b2 = tf.get_variable('b2', shape=[1], dtype=tf.float32,
                             initializer=tf.constant_initializer(0.0))
        # wgan不适用sigmoid
        logits = tf.add(tf.matmul(fc1, w2), b2, name='logits')
 
        return logits
##---------------------------------------------------------generator
def generator_tf_wgan(input, random_dim, is_train, reuse=False):
    with tf.variable_scope('generator') as scope:
        if reuse:
            scope.reuse_variables()
        w1 = tf.get_variable('w1', shape=[random_dim, 4 * 4 * 512], dtype=tf.float32,
                             initializer=tf.truncated_normal_initializer(stddev=0.02))
        b1 = tf.get_variable('b1', shape=[512 * 4 * 4], dtype=tf.float32,
                             initializer=tf.constant_initializer(0.0))
        flat_conv1 = tf.add(tf.matmul(input, w1), b1, name='flat_conv1')
 
        # 4*4*512
        conv1 = tf.reshape(flat_conv1, shape=[-1, 4, 4, 512], name='conv1')
        bn1 = tf.contrib.layers.batch_norm(conv1, is_training=is_train, epsilon=1e-5, decay=0.9,
                                           updates_collections=None, scope='bn1')
        act1 = tf.nn.relu(bn1, name='act1')
 
        # 8*8*256
        conv2 = tf.layers.conv2d_transpose(act1, 256, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv2')
        bn2 = tf.contrib.layers.batch_norm(conv2, is_training=is_train, epsilon=1e-5, decay=0.9,
                                           updates_collections=None, scope='bn2')
        act2 = tf.nn.relu(bn2, name='act2')
 
        # 16*16*128
        conv3 = tf.layers.conv2d_transpose(act2, 128, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv3')
        bn3 = tf.contrib.layers.batch_norm(conv3, is_training=is_train, epsilon=1e-5, decay=0.9,
                                           updates_collections=None, scope='bn3')
        act3 = tf.nn.relu(bn3, name='act3')
 
        # 32*32*64
        conv4 = tf.layers.conv2d_transpose(act3, 64, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv4')
        bn4 = tf.contrib.layers.batch_norm(conv4, is_training=is_train, epsilon=1e-5, decay=0.9,
                                           updates_collections=None, scope='bn4')
        act4 = tf.nn.relu(bn4, name='act4')
 
        # 64*64*32
        conv5 = tf.layers.conv2d_transpose(act4, 32, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv5')
        bn5 = tf.contrib.layers.batch_norm(conv5, is_training=is_train, epsilon=1e-5, decay=0.9,
                                           updates_collections=None, scope='bn5')
        act5 = tf.nn.relu(bn5, name='act5')
 
        # 128*128*3
        conv6 = tf.layers.conv2d_transpose(act5, image_channel, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv6')
 
        act6 = tf.nn.tanh(conv6, name='act6')
 
        return act6

In [ ]:
#----------------------------------------------------------
#
#WGAN model definition
#
#----------------------------------------------------------
## pytorch
"""
Discriminator and Generator implementation from DCGAN paper,
with removed Sigmoid() as output from Discriminator (and therefor
it should be called critic)
"""


class WGAN_Discriminator(nn.Module):
    def __init__(self, channels_img, features_d):
        super(WGAN_Discriminator, self).__init__()
        
        self.disc = nn.Sequential(  
            # input: N x channels_img x 64 x 64
            nn.Conv2d(channels_img, features_d, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            
            # _block(in_channels, out_channels, kernel_size, stride, padding)
            self._block(features_d, features_d * 2, 4, 2, 1),
            self._block(features_d * 2, features_d * 4, 4, 2, 1),
            self._block(features_d * 4, features_d * 8, 4, 2, 1),
            # After all _block img output is 4x4 (Conv2d below makes into 1x1)
            nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0),
        )
        # ??? output shape? 

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,            ),
            #???
            nn.InstanceNorm2d(out_channels, affine=True),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.disc(x)


class WGAN_Generator(nn.Module):
    def __init__(self, channels_noise, channels_img, features_g):
        super(WGAN_Generator, self).__init__()
        self.net = nn.Sequential(
            # Input: N x channels_noise x 1 x 1
            self._block(channels_noise, features_g * 16, 4, 1, 0),  # img: 4x4
            self._block(features_g * 16, features_g * 8, 4, 2, 1),  # img: 8x8
            self._block(features_g * 8, features_g * 4, 4, 2, 1),  # img: 16x16
            self._block(features_g * 4, features_g * 2, 4, 2, 1),  # img: 32x32
            nn.ConvTranspose2d(
                features_g * 2, channels_img, kernel_size=4, stride=2, padding=1
            ),
            # Output: N x channels_img x 64 x 64
            nn.Tanh(),
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.net(x)



## 4 Pix2Pix GAN

### Introduction

Image-to-image translation is the controlled conversion of a given source image to a target image.

An example might be the conversion of black and white photographs to color photographs.

Image-to-image translation is a challenging problem and often requires specialized models and loss functions for a given translation task or dataset.

The Pix2Pix GAN is a general approach for image-to-image translation. It is based on the conditional generative adversarial network, where a target image is generated, conditional on a given input image. In this case, the Pix2Pix GAN changes the loss function so that the generated image is both plausible in the content of the target domain, and is a plausible translation of the input image.

Referance:https://machinelearningmastery.com/a-gentle-introduction-to-pix2pix-generative-adversarial-network/

Paper link: \\
https://arxiv.org/abs/1611.07004

### Implementation

In [ ]:
## tensorflow
#----------------------------------------------- discriminator
class Discriminator_tf_p2p(object):
    def __init__(self, inputs, is_training, stddev=0.02, center=True, scale=True, reuse=None):
        self._is_training = is_training
        self._stddev = stddev

        with tf.variable_scope('D', initializer=tf.truncated_normal_initializer(stddev=self._stddev), reuse=reuse):
            self._center = center
            self._scale = scale
            self._prob = 0.5 # constant from pix2pix paper
            self._inputs = inputs
            self._discriminator = self._build_discriminator(inputs, reuse=reuse)

    def _build_layer(self, name, inputs, k, bn=True, use_dropout=False):
        layer = dict()
        with tf.variable_scope(name):
            layer['filters'] = tf.get_variable('filters', [4, 4, get_shape(inputs)[-1], k])
            layer['conv'] = tf.nn.conv2d(inputs, layer['filters'], strides=[1, 2, 2, 1], padding='SAME')
            layer['bn'] = batch_norm(layer['conv'], center=self._center, scale=self._scale, training=self._is_training) if bn else layer['conv']
            layer['dropout'] = tf.nn.dropout(layer['bn'], self._prob) if use_dropout else layer['bn']
            layer['fmap'] = lkrelu(layer['dropout'], slope=0.2)
        return layer

    def _build_discriminator(self, inputs, reuse=None):
        discriminator = dict()

        # C64-C128-C256-C512 -> PatchGAN
        discriminator['l1'] = self._build_layer('l1', inputs, 64, bn=False)
        discriminator['l2'] = self._build_layer('l2', discriminator['l1']['fmap'], 128)
        discriminator['l3'] = self._build_layer('l3', discriminator['l2']['fmap'], 256)
        discriminator['l4'] = self._build_layer('l4', discriminator['l3']['fmap'], 512)
        with tf.variable_scope('l5'):
            l5 = dict()
            l5['filters'] = tf.get_variable('filters', [4, 4, get_shape(discriminator['l4']['fmap'])[-1], 1])
            l5['conv'] = tf.nn.conv2d(discriminator['l4']['fmap'], l5['filters'], strides=[1, 1, 1, 1], padding='SAME')
            l5['bn'] = batch_norm(l5['conv'], center=self._center, scale=self._scale, training=self._is_training)
            l5['fmap'] = tf.nn.sigmoid(l5['bn'])
            discriminator['l5'] = l5
        return discriminator
#----------------------------------------------- generator
class Generator_tf_p2p(object):
    def __init__(self, inputs, is_training, ochan, stddev=0.02, center=True, scale=True, reuse=None):
        self._is_training = is_training
        self._stddev = stddev
        self._ochan = ochan
        with tf.variable_scope('G', initializer=tf.truncated_normal_initializer(stddev=self._stddev), reuse=reuse):
            self._center = center
            self._scale = scale
            self._prob = 0.5 # constant from pix2pix paper
            self._inputs = inputs
            self._encoder = self._build_encoder(inputs)
            self._decoder = self._build_decoder(self._encoder)

    def _build_encoder_layer(self, name, inputs, k, bn=True, use_dropout=False):
        layer = dict()
        with tf.variable_scope(name):
            layer['filters'] = tf.get_variable('filters', [4, 4, get_shape(inputs)[-1], k])
            layer['conv'] = tf.nn.conv2d(inputs, layer['filters'], strides=[1, 2, 2, 1], padding='SAME')
            layer['bn'] = batch_norm(layer['conv'], center=self._center, scale=self._scale, training=self._is_training) if bn else layer['conv']
            layer['dropout'] = tf.nn.dropout(layer['bn'], self._prob) if use_dropout else layer['bn']
            layer['fmap'] = lkrelu(layer['dropout'], slope=0.2)
        return layer

    def _build_encoder(self, inputs):
        encoder = dict()

        # C64-C128-C256-C512-C512-C512-C512-C512
        with tf.variable_scope('encoder'):
            encoder['l1'] = self._build_encoder_layer('l1', inputs, 64, bn=False)
            encoder['l2'] = self._build_encoder_layer('l2', encoder['l1']['fmap'], 128)
            encoder['l3'] = self._build_encoder_layer('l3', encoder['l2']['fmap'], 256)
            encoder['l4'] = self._build_encoder_layer('l4', encoder['l3']['fmap'], 512)
            encoder['l5'] = self._build_encoder_layer('l5', encoder['l4']['fmap'], 512)
            encoder['l6'] = self._build_encoder_layer('l6', encoder['l5']['fmap'], 512)
            encoder['l7'] = self._build_encoder_layer('l7', encoder['l6']['fmap'], 512)
            encoder['l8'] = self._build_encoder_layer('l8', encoder['l7']['fmap'], 512)
        return encoder

    def _build_decoder_layer(self, name, inputs, output_shape_from,use_dropout=False):
        layer = dict()

        with tf.variable_scope(name):
            output_shape = tf.shape(output_shape_from)
            layer['filters'] = tf.get_variable('filters', [4, 4, get_shape(output_shape_from)[-1], get_shape(inputs)[-1]])
            layer['conv'] = tf.nn.conv2d_transpose(inputs, layer['filters'], output_shape=output_shape, strides=[1, 2, 2, 1], padding='SAME')
            layer['bn'] = batch_norm(tf.reshape(layer['conv'], output_shape), center=self._center, scale=self._scale, training=self._is_training)
            layer['dropout'] = tf.nn.dropout(layer['bn'], self._prob) if use_dropout else layer['bn']
            layer['fmap'] = tf.nn.relu(layer['dropout'])
        return layer

    def _build_decoder(self, encoder):
        decoder = dict()

        # CD512-CD1024-CD1024-C1024-C1024-C512-C256-C128
        with tf.variable_scope('decoder'): # U-Net
            decoder['dl1'] = self._build_decoder_layer('dl1', encoder['l8']['fmap'], output_shape_from=encoder['l7']['fmap'], use_dropout=True)

            # fmap_concat represent skip connections
            fmap_concat = tf.concat([decoder['dl1']['fmap'], encoder['l7']['fmap']], axis=3)
            decoder['dl2'] = self._build_decoder_layer('dl2', fmap_concat, output_shape_from=encoder['l6']['fmap'], use_dropout=True)

            fmap_concat = tf.concat([decoder['dl2']['fmap'], encoder['l6']['fmap']], axis=3)
            decoder['dl3'] = self._build_decoder_layer('dl3', fmap_concat, output_shape_from=encoder['l5']['fmap'], use_dropout=True)

            fmap_concat = tf.concat([decoder['dl3']['fmap'], encoder['l5']['fmap']], axis=3)
            decoder['dl4'] = self._build_decoder_layer('dl4', fmap_concat, output_shape_from=encoder['l4']['fmap'])

            fmap_concat = tf.concat([decoder['dl4']['fmap'], encoder['l4']['fmap']], axis=3)
            decoder['dl5'] = self._build_decoder_layer('dl5', fmap_concat, output_shape_from=encoder['l3']['fmap'])

            fmap_concat = tf.concat([decoder['dl5']['fmap'], encoder['l3']['fmap']], axis=3)
            decoder['dl6'] = self._build_decoder_layer('dl6', fmap_concat, output_shape_from=encoder['l2']['fmap'])

            fmap_concat = tf.concat([decoder['dl6']['fmap'], encoder['l2']['fmap']], axis=3)
            decoder['dl7'] = self._build_decoder_layer('dl7', fmap_concat, output_shape_from=encoder['l1']['fmap'])

            fmap_concat = tf.concat([decoder['dl7']['fmap'], encoder['l1']['fmap']], axis=3)
            decoder['dl8'] = self._build_decoder_layer('dl8', fmap_concat, output_shape_from=self._inputs)

            with tf.variable_scope('cl9'):
                cl9 = dict()
                cl9['filters'] = tf.get_variable('filters', [4, 4, get_shape(decoder['dl8']['fmap'])[-1], self._ochan])
                cl9['conv'] =  tf.nn.conv2d(decoder['dl8']['fmap'], cl9['filters'], strides=[1, 1, 1, 1], padding='SAME')
                cl9['fmap'] = tf.nn.tanh(cl9['conv'])
                decoder['cl9'] = cl9
        return decoder



In [ ]:
## pytorch
### generator
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, act="relu", use_dropout=False):
        super(Block, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, 2, 1, bias=False, padding_mode="reflect")
            if down
            else nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU() if act == "relu" else nn.LeakyReLU(0.2),
        )

        self.use_dropout = use_dropout
        self.dropout = nn.Dropout(0.5)
        self.down = down

    def forward(self, x):
        x = self.conv(x)
        return self.dropout(x) if self.use_dropout else x


class P2P_Generator(nn.Module):
    def __init__(self, in_channels=3, features=64):
        super().__init__()
        self.initial_down = nn.Sequential(
            nn.Conv2d(in_channels, features, 4, 2, 1, padding_mode="reflect"),
            nn.LeakyReLU(0.2),
        )
        self.down1 = Block(features, features * 2, down=True, act="leaky", use_dropout=False)
        self.down2 = Block(
            features * 2, features * 4, down=True, act="leaky", use_dropout=False
        )
        self.down3 = Block(
            features * 4, features * 8, down=True, act="leaky", use_dropout=False
        )
        self.down4 = Block(
            features * 8, features * 8, down=True, act="leaky", use_dropout=False
        )
        self.down5 = Block(
            features * 8, features * 8, down=True, act="leaky", use_dropout=False
        )
        self.down6 = Block(
            features * 8, features * 8, down=True, act="leaky", use_dropout=False
        )
        self.bottleneck = nn.Sequential(
            nn.Conv2d(features * 8, features * 8, 4, 2, 1), nn.ReLU()
        )

        self.up1 = Block(features * 8, features * 8, down=False, act="relu", use_dropout=True)
        self.up2 = Block(
            features * 8 * 2, features * 8, down=False, act="relu", use_dropout=True
        )
        self.up3 = Block(
            features * 8 * 2, features * 8, down=False, act="relu", use_dropout=True
        )
        self.up4 = Block(
            features * 8 * 2, features * 8, down=False, act="relu", use_dropout=False
        )
        self.up5 = Block(
            features * 8 * 2, features * 4, down=False, act="relu", use_dropout=False
        )
        self.up6 = Block(
            features * 4 * 2, features * 2, down=False, act="relu", use_dropout=False
        )
        self.up7 = Block(features * 2 * 2, features, down=False, act="relu", use_dropout=False)
        self.final_up = nn.Sequential(
            nn.ConvTranspose2d(features * 2, in_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh(),
        )

    def forward(self, x):
        d1 = self.initial_down(x)
        d2 = self.down1(d1)
        d3 = self.down2(d2)
        d4 = self.down3(d3)
        d5 = self.down4(d4)
        d6 = self.down5(d5)
        d7 = self.down6(d6)
        bottleneck = self.bottleneck(d7)
        up1 = self.up1(bottleneck)
        up2 = self.up2(torch.cat([up1, d7], 1))
        up3 = self.up3(torch.cat([up2, d6], 1))
        up4 = self.up4(torch.cat([up3, d5], 1))
        up5 = self.up5(torch.cat([up4, d4], 1))
        up6 = self.up6(torch.cat([up5, d3], 1))
        up7 = self.up7(torch.cat([up6, d2], 1))
        return self.final_up(torch.cat([up7, d1], 1))

### discriminator

class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(CNNBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(
                in_channels, out_channels, 4, stride, 1, bias=False, padding_mode="reflect"
            ),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.conv(x)


class P2P_Discriminator(nn.Module):
    def __init__(self, in_channels=3, features=[64, 128, 256, 512]):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(
                in_channels * 2,
                features[0],
                kernel_size=4,
                stride=2,
                padding=1,
                padding_mode="reflect",
            ),
            nn.LeakyReLU(0.2),
        )

        layers = []
        in_channels = features[0]
        for feature in features[1:]:
            layers.append(
                CNNBlock(in_channels, feature, stride=1 if feature == features[-1] else 2),
            )
            in_channels = feature

        layers.append(
            nn.Conv2d(
                in_channels, 1, kernel_size=4, stride=1, padding=1, padding_mode="reflect"
            ),
        )

        self.model = nn.Sequential(*layers)

    def forward(self, x, y):
        x = torch.cat([x, y], dim=1)
        x = self.initial(x)
        x = self.model(x)
        return x

## Training GAN models

### Save Checkpoints

In [ ]:
def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)

### Weights Initialization

In [ ]:
def initialize_weights(model):
    # Initializes weights according to the DCGAN paper
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)

### Optimization

In [ ]:
##################################### select optimalization
opt = ['Adam', 'RMS'] # 0, 1
def Sel_Opt(opt, gen, disc):
  if opt == 0:
    opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
    opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
    return opt_gen, opt_disc
  elif opt == 1:
    opt_gen = optim.RMSprop(gen.parameters(), lr=LEARNING_RATE)
    opt_critic = optim.RMSprop(disc.parameters(), lr=LEARNING_RATE)
    return opt_gen, opt_critic 

### Loss function

In [ ]:
criterion =  nn.BCELoss()
criterionP2P = nn.BCEWithLogitsLoss()
l1_loss = nn.L1Loss()

### Train functions

In [ ]:
##################################### train discriminator and generator
def trainDisc(real,fake,disc, opt_disc):
  print(fake.shape)
  print(real.shape)
  print(ok)
  disc_real = disc(real).reshape(-1)
  loss_disc_real = criterion(disc_real, torch.ones_like(disc_real))
  disc_fake = disc(fake.detach()).reshape(-1)
  loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
  loss_disc = (loss_disc_real + loss_disc_fake) / 2
  disc.zero_grad()
  loss_disc.backward()
  opt_disc.step()
  return loss_disc

def trainGen(fake, disc, gen, opt_gen):
  output = disc(fake).reshape(-1)
  loss_gen = criterion(output, torch.ones_like(output))
  gen.zero_grad()
  loss_gen.backward()
  opt_gen.step()
  return loss_gen

def trainDisc_WGAN(real, CRITIC_ITERATIONS, BATCH_SIZE, gen, disc, opt_disc):
  for _ in range(CRITIC_ITERATIONS):
    noise = torch.randn(BATCH_SIZE, Z_DIM, 1, 1).to(device)
    fake = gen(noise)
    critic_real = disc(real).reshape(-1)
    critic_fake = disc(fake).reshape(-1)
    loss_critic = -(torch.mean(critic_real) - torch.mean(critic_fake))
    disc.zero_grad()
    loss_critic.backward(retain_graph=True)
    opt_disc.step()

    # clip critic weights between -0.01, 0.01
    for p in disc.parameters():
        p.data.clamp_(-WEIGHT_CLIP, WEIGHT_CLIP)
    return loss_critic, fake

def trainGen_WGAN(fake, gen, opt_gen, disc):
  gen_fake = disc(fake).reshape(-1)
  loss_gen = -torch.mean(gen_fake)
  gen.zero_grad()
  loss_gen.backward()
  opt_gen.step()
  return loss_gen

def trainDisc_P2P(disc, gen, opt_disc, criterionP2P, d_scaler, real, y):
  # Train Discriminator
      with torch.cuda.amp.autocast():
          y_fake = gen(real)
          D_real = disc(real, y)
          D_real_loss = criterionP2P(D_real, torch.ones_like(D_real))
          D_fake = disc(real, y_fake.detach())
          D_fake_loss = criterionP2P(D_fake, torch.zeros_like(D_fake))
          D_loss = (D_real_loss + D_fake_loss) / 2

      disc.zero_grad()
      d_scaler.scale(D_loss).backward()
      d_scaler.step(opt_disc)
      d_scaler.update()
      return disc, opt_disc, D_loss, y_fake



def trainGen_P2P(disc, gen, opt_gen, l1_loss, criterionP2P, g_scaler, real, y, y_fake):
      # # Train Discriminator
      # with torch.cuda.amp.autocast():
      #     y_fake = gen(real)
      #     D_real = disc(real, y)
      #     D_real_loss = bce(D_real, torch.ones_like(D_real))
      #     D_fake = disc(real, y_fake.detach())
      #     D_fake_loss = bce(D_fake, torch.zeros_like(D_fake))
      #     D_loss = (D_real_loss + D_fake_loss) / 2

      # disc.zero_grad()
      # d_scaler.scale(D_loss).backward()
      # d_scaler.step(opt_disc)
      # d_scaler.update()

      # Train generator
      with torch.cuda.amp.autocast():
          D_fake = disc(real, y_fake)
          G_fake_loss = criterionP2P(D_fake, torch.ones_like(D_fake))
          L1 = l1_loss(y_fake, y) * 100
          G_loss = G_fake_loss + L1

      opt_gen.zero_grad()
      g_scaler.scale(G_loss).backward()
      g_scaler.step(opt_gen)
      g_scaler.update()
      return gen, opt_gen, G_loss



# def trainDisc_P2P():

def tf_Train_DCdisc(loss_fn, batch_size, discriminator, fake, real, opt_disc):
  with tf.GradientTape() as disc_tape:
    print(real.shape)
    real = tf.image.resize(real, [64, 64])
    print(discriminator(real).shape)
    print(fake.shape)
    print((tf.ones((batch_size, 1))).shape)
    loss_disc_real = loss_fn(tf.ones((batch_size, 1)), discriminator(real))
    loss_disc_fake = loss_fn(tf.zeros(batch_size, 1), discriminator(fake))
    loss_disc = (loss_disc_real + loss_disc_fake)/2

    grads = disc_tape.gradient(loss_disc, discriminator.trainable_weights)
    opt_disc.apply_gradients(
        zip(grads, discriminator.trainable_weights)
    )

def tf_Train_DCgen(generator, discriminator, batch_size, loss_fn, random_latent_vectors, opt_gen):
  with tf.GradientTape() as gen_tape:
    fake = generator(random_latent_vectors)
    output = discriminator(fake)
    loss_gen = loss_fn(tf.ones(batch_size, 1), output)

    grads = gen_tape.gradient(loss_gen, generator.trainable_weights)
    opt_gen.apply_gradients(
        zip(grads, generator.trainable_weights)
    )


### plot functions

In [ ]:
################################# plot
def pltLoss(epoch, dataloader, batch_idx, loss_disc, loss_gen, fixed_noise, gen, real, GANID):
  print( f"Epoch [{epoch}/{NUM_EPOCHS}] Batch {batch_idx}/{len(dataloader)} \
                  Loss D: {loss_disc:.4f}, loss G: {loss_gen:.4f}")

  with torch.no_grad():
      if (GANID == 3) or(GANID == 1)or(GANID == 2):
        fake = gen(fixed_noise)
        # print()
        # fake = gen(y).reshape(-1, 1, 28, 28)
        # print(y.shape)
        # print('================')
        # print(real.shape)
        # print(ok)
      else:
        fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
        # print(fake.shape)
        # print(ok)
        real = real.reshape(-1, 1, 28, 28)
      # take out (up to) 32 examples
      print('%%%%%%%%%%%%%%%', real.shape)
      print('%%%%%%%%%%%%%%%%', fake.shape)
      img_grid_real = torchvision.utils.make_grid(
          real[:1], normalize=True
      )
      img_grid_fake = torchvision.utils.make_grid(
          fake[:1], normalize=True
      )
      
      plt.figure()
      plt.xticks([])
      plt.yticks([])
      plt.grid(False)
      print('img grid real--------------------',img_grid_real[0].shape)
      imgreal = img_grid_fake[0].cpu().numpy()
      print(imgreal*255)
      plt.imshow(cv2.cvtColor(imgreal, cv2.COLOR_BGR2RGB))
      plt.show()
      imgfake = (img_grid_fake[0].cpu().numpy())*255
      plt.imshow(cv2.cvtColor(imgfake, cv2.COLOR_BGR2RGB))
      plt.show()
      # print('$$$$$$$$$$$$', img_grid_real.shape)
      # plt.figure(figsize=(10,10))
      # # plt.imshow(img_grid_real)

      # for i in range(25):
      #     plt.subplot(5,5,i+1)
      #     plt.xticks([])
      #     plt.yticks([])
      #     plt.grid(False)
      #     print(img_grid_real[i].shape)
      #     imgreal = img_grid_real[i].cpu().numpy()
      #     print(imgreal.shape)
      #     plt.imshow(cv2.cvtColor(imgreal, cv2.COLOR_BGR2RGB))

      #     # The CIFAR labels happen to be arrays, 
      #     # which is why you need the extra index
      # plt.show()
      # plt.figure(figsize=(10,10))
      # # plt.imshow(img_grid_real)
      # for i in range(25):
      #     plt.subplot(5,5,i+1)
      #     plt.xticks([])
      #     plt.yticks([])
      #     plt.grid(False)
      #     #plt.imshow(x_train[i])
      #     imgfake = img_grid_fake[i].cpu().numpy()
      #     plt.imshow(cv2.cvtColor(imgfake, cv2.COLOR_BGR2RGB))

      #     # The CIFAR labels happen to be arrays, 
      #     # which is why you need the extra index
      # plt.show()

      # writer_real.add_image("Real", img_grid_real, global_step=step)
      # writer_fake.add_image("Fake", img_grid_fake, global_step=step)

## Assemble training

In [ ]:
# dataset = keras.preprocessing.image_dataset_from_directory(
#     directory="celeb_dataset", label_mode=None, image_size=(64, 64), batch_size=32,
#     shuffle=True
# ).map(lambda x: x/255.0)

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

In [ ]:
def trainFunc(IMAGE_SIZE):
  if usePt == 0:
    if GANID == 1: # DCGAN
      opt_gen = keras.optimizers.Adam(1e-4)
      opt_disc = keras.optimizers.Adam(1e-4)
      loss_fn = keras.losses.BinaryCrossentropy()
      gen = generator_tf_dc
      disc = discriminator_tf_dc
    elif GANID == 2: # WGAN
      gen = generator_tf_wgan(input, random_dim, is_train, reuse=False)
      disc = discriminator_tf_wgan
      opt_gen = 
      opt_disc = 
      loss_fn_g = 
      loss_fn_d = 


      with tf.variable_scope('input'):
        # 模型中的输入数据部分
        real_image = tf.placeholder(tf.float32, shape=[None, IMAGE_SIZE, IMAGE_SIZE, 1], name='real_image')
        random_input = tf.placeholder(tf.float32, shape=[None, Z_DIM], name='rand_input')
        is_train = tf.placeholder(tf.bool, name='is_train')


    for epoch in range(10):
      for idx, (real, y_train) in enumerate(train_dataset):
        batch_size = real.shape[0]
        print(batch_size)
        random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))
        fake = gen(random_latent_vectors)
        # if idx % 100 == 0:
        #   img = keras.preprocessing.image.array_to_img(fake[0])
        #   img.save(f"generated_images/generated_img{epoch}_{idx}_.png")

        ### Train Discriminator: max log(D(x)) + log(1 - D(G(z))
        tf_Train_DCdisc(loss_fn, batch_size, disc, fake, real, opt_disc)
        ### Train Generator min log(1 - D(G(z)) <-> max log(D(G(z))
        tf_Train_DCgen(gen, disc, batch_size, loss_fn, random_latent_vectors, opt_gen)
        print('finshied------------------')







  else:
    if GANID == 0:
      IMAGE_SIZE = 28 * 28 * 1
      disc = SimpleDiscriminator(IMAGE_SIZE).to(device)
      gen  = SimpleGenerator(Z_DIM, IMAGE_SIZE).to(device)
      fixed_noise = torch.randn(BATCH_SIZE, Z_DIM).to(device)
    elif GANID == 1:
      IMAGE_SIZE = IMAGE_SIZE
      gen = DCGAN_Generator(Z_DIM, CHANNELS_IMG, FEATURES_GEN).to(device)
      disc = DCGAN_Discriminator(CHANNELS_IMG, FEATURES_DISC).to(device)
      fixed_noise = torch.randn(BATCH_SIZE, Z_DIM, 1, 1).to(device)
    elif GANID == 2:
      gen = WGAN_Generator(Z_DIM, CHANNELS_IMG, FEATURES_GEN).to(device)
      disc = WGAN_Discriminator(CHANNELS_IMG, FEATURES_DISC).to(device)
      fixed_noise = torch.randn(BATCH_SIZE, Z_DIM, 1, 1).to(device)
    elif GANID == 3:
      gen = P2P_Generator(in_channels=3, features=64).to(device)
      disc = P2P_Discriminator(in_channels=3).to(device)
      # fixed_noise = torch.randn(BATCH_SIZE, Z_DIM, 1, 1).to(device)
      
      
    initialize_weights(gen)
    initialize_weights(disc)
    dataset = datasetReturn(GANID)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    if GANID == 3:
      dataset = MapDataset(root_dir='/content/drive/MyDrive/Colab Notebooks/datasets/maps/train/')
      dataloader = DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=True,
        )
    opt_gen, opt_disc = Sel_Opt(0, gen, disc) # optimal function selection

    
    writer_real = SummaryWriter(f"logs/real")
    writer_fake = SummaryWriter(f"logs/fake")
    step = 0

    gen.train()
    disc.train()
    ################################################# train start
    for epoch in range(NUM_EPOCHS):
      # Target labels not needed! <3 unsupervised
      for batch_idx, (real, y) in enumerate(dataloader):
        real = real.to(device)
        y = y.to(device)
        # print(real)
        # print(fake1)
        # print(ok)
        # print('real',real.shape)
        if GANID == 0:
          real = real.view(-1, 784).to(device)
          noise = torch.randn(batch_size, Z_DIM).to(device)
          # print('noise',noise.shape)
          fake = gen(noise)
          # print('',fake.shape)
        elif (GANID == 1):
          noise = torch.randn(BATCH_SIZE, Z_DIM, 1, 1).to(device)
          fake = gen(noise)
          print(real.shape)
        elif (GANID == 3):
          g_scaler = torch.cuda.amp.GradScaler()
          d_scaler = torch.cuda.amp.GradScaler()
          

        if GANID == 2:
          loss_disc, fake = trainDisc_WGAN(real, CRITIC_ITERATIONS, BATCH_SIZE, gen, disc, opt_disc)
          loss_gen = trainGen_WGAN(fake, gen, opt_gen, disc)
        elif (GANID == 0)or(GANID == 1):
          loss_disc = trainDisc(real, fake, disc, opt_disc)
          loss_gen = trainGen(fake, disc, gen, opt_gen)
        elif (GANID == 3):
          disc, opt_disc, loss_disc, y_fake = trainDisc_P2P(disc, gen, opt_disc, criterionP2P, d_scaler, real, y)
          gen, opt_gen, loss_gen = trainGen_P2P(disc, gen, opt_gen, l1_loss, criterionP2P, g_scaler, real, y, y_fake)
          fixed_noise = y

        # Print losses occasionally and print to tensorboard
        if batch_idx % 100 == 0:
          pltLoss(epoch, dataloader, batch_idx, loss_disc, loss_gen, fixed_noise, gen, real, GANID)
          step += 1
      if (epoch % 100 == 0) or (epoch == NUM_EPOCHS-1):
        save_checkpoint(gen, opt_gen, filename="gen_checkpoint_"+str(epoch) +".pth.tar")
        save_checkpoint(disc, opt_disc, filename = "disc_checkpoint_"+str(epoch) +".pth.tar")


In [ ]:
trainFunc(IMAGE_SIZE)

# More resources:

* 3Blue1Brown Neural Network [video tutorials](https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi) 
* Deep Learning Video Lectures by Prof. Andreas Maier [Winter 20/21](https://www.youtube.com/watch?v=SCFToE1vM2U&list=PLpOGQvPCDQzvJEPFUQ3mJz72GJ95jyZTh)
* Some of the code in this notebook is taken from [here](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch)
* Calculating number of parameters in [CNN](https://towardsdatascience.com/understanding-and-calculating-the-number-of-parameters-in-convolution-neural-networks-cnns-fc88790d530d)
* Some of the code in this notebook is taken from [here](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/cnn.ipynb)
* https://imerit.net/blog/top-13-machine-learning-image-classification-datasets-all-pbm/
* https://nihcc.app.box.com/v/ChestXray-NIHCC
* https://www.kaggle.com/xhlulu/recursion-cellular-image-classification-224-jpg
* https://www.tensorflow.org/datasets/catalog/patch_camelyon
* https://www.youtube.com/playlist?list=PLhhyoLH6IjfwIp8bZnzX8QR30TRcHO8Va
* https://developers.google.com/machine-learning/gan/gan_structure
* https://machinelearningmastery.com/impressive-applications-of-generative-adversarial-networks/